# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
import os
import pandas as pd


### 데이터 읽어오기


In [7]:
RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/trim_train_data.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/trim_test_data.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 464 entries, Wip Line_Dam to target
dtypes: float64(342), int64(88), object(34)
memory usage: 143.4+ MB


In [8]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 465 entries, Set ID to target
dtypes: float64(400), int64(31), object(34)
memory usage: 61.6+ MB


In [12]:
# # excel 데이터의 각 열에 대한 값의 개수 출력 -> 변수명 확인용도
# for col in train_data.columns:
#     counts = train_data[col].value_counts()
#     print(f"Value counts for column {col}:")
#     print(counts)
#     print()

In [9]:
# target 열을 임시로 분리
target_train = train_data['target']
target_test = test_data['target']

# 모든 값이 NaN인 열 제거
train_data = train_data.dropna(axis=1, how='all')
test_data = test_data.dropna(axis=1, how='all')

# target 열을 다시 결합
train_data['target'] = target_train
test_data['target'] = target_test

# 제거 후 데이터 확인
print("All columns with only NaN values have been removed, except for the target column.")

All columns with only NaN values have been removed, except for the target column.


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 186 entries, Wip Line_Dam to target
dtypes: float64(64), int64(88), object(34)
memory usage: 57.5+ MB


In [15]:
# # excel 데이터의 각 열에 대한 값의 개수 출력 -> 변수명 확인용도
# for col in train_data.columns:
#     counts = train_data[col].value_counts()
#     print(f"Value counts for column {col}:")
#     print(counts)
#     print()

In [11]:
wip_line_columns = train_data.filter(like='Wip Line').columns

new_df = train_data.filter(items=wip_line_columns)

new_df.value_counts()

Wip Line_Dam  Wip Line_AutoClave  Wip Line_Fill1  Wip Line_Fill2
IVI-OB6       IVI-OB6             IVI-OB6         IVI-OB6           40506
Name: count, dtype: int64

In [12]:
train_data.drop(columns=wip_line_columns, inplace=True)
test_data.drop(columns=wip_line_columns, inplace=True)

In [13]:
Process_Desc_col = train_data.filter(like='Process Desc').columns

new2_df = train_data.filter(items=Process_Desc_col)

new2_df.value_counts()

Process Desc._Dam  Process Desc._AutoClave  Process Desc._Fill1  Process Desc._Fill2
Dam Dispenser      Auto Clave Out           Fill1 Dispenser      Fill2 Dispenser        40506
Name: count, dtype: int64

In [14]:
train_data = train_data.drop(columns=['Process Desc._Dam', 'Process Desc._AutoClave', 'Process Desc._Fill1', 'Process Desc._Fill2'])
test_data = test_data.drop(columns=['Process Desc._Dam', 'Process Desc._AutoClave', 'Process Desc._Fill1', 'Process Desc._Fill2'])

In [15]:
# Equipment로 시작하는 열 필터링
Equipment_col = train_data.filter(like='Equipment').columns
new3_df = train_data.filter(items=Equipment_col)

# target 변수 추가
new3_df['target'] = train_data['target']

# 그룹별로 target 변수의 비율 계산
target_ratio = new3_df.groupby(list(Equipment_col))['target'].value_counts(normalize=True).unstack().fillna(0)

# AbNormal 비율 출력
abnormal_ratio = target_ratio.get('AbNormal', 0)
print("각 그룹의 abnormal_ratio:")
print(abnormal_ratio)

# 각 그룹에 값의 갯수 출력
group_counts = new3_df.groupby(list(Equipment_col)).size()
print("각 그룹의 값의 갯수:")
print(group_counts)

각 그룹의 abnormal_ratio:
Equipment_Dam     Equipment_AutoClave  Equipment_Fill1     Equipment_Fill2   
Dam dispenser #1  Auto Clave Out       Fill1 dispenser #1  Fill2 dispenser #1    0.058614
                                                           Fill2 dispenser #2    1.000000
                                       Fill1 dispenser #2  Fill2 dispenser #2    1.000000
Dam dispenser #2  Auto Clave Out       Fill1 dispenser #1  Fill2 dispenser #1    1.000000
                                       Fill1 dispenser #2  Fill2 dispenser #1    1.000000
                                                           Fill2 dispenser #2    0.054977
Name: AbNormal, dtype: float64
각 그룹의 값의 갯수:
Equipment_Dam     Equipment_AutoClave  Equipment_Fill1     Equipment_Fill2   
Dam dispenser #1  Auto Clave Out       Fill1 dispenser #1  Fill2 dispenser #1    25011
                                                           Fill2 dispenser #2        6
                                       Fill1 dispenser #2  Fill2

In [16]:
# Equipment로 시작하는 열 필터링
Equipment_col = train_data.filter(like='Equipment').columns
Equipment_col2 = test_data.filter(like='Equipment').columns

new3_df = train_data.filter(items=Equipment_col)
new3_df2 = test_data.filter(items=Equipment_col2)

# Equipment_same_num 파생변수 생성
def determine_equipment_same_num(row):
    if (row['Equipment_Dam'] == 'Dam dispenser #1' and row['Equipment_AutoClave'] == 'Auto Clave Out' and 
        row['Equipment_Fill1'] == 'Fill1 dispenser #1' and row['Equipment_Fill2'] == 'Fill2 dispenser #1') or \
       (row['Equipment_Dam'] == 'Dam dispenser #2' and row['Equipment_AutoClave'] == 'Auto Clave Out' and 
        row['Equipment_Fill1'] == 'Fill1 dispenser #2' and row['Equipment_Fill2'] == 'Fill2 dispenser #2'):
        return 1
    else:
        return 0

train_data['Equipment_same_num'] = new3_df.apply(determine_equipment_same_num, axis=1)
test_data['Equipment_same_num'] = new3_df2.apply(determine_equipment_same_num, axis=1)

train_data = train_data.drop(columns=['Equipment_Dam', 'Equipment_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2'])
test_data = test_data.drop(columns=['Equipment_Dam', 'Equipment_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2'])


In [17]:
Process_Desc_col = train_data.filter(like='Model.Suffix').columns

new2_df = train_data.filter(items=Process_Desc_col)

new2_df.value_counts()

Model.Suffix_Dam  Model.Suffix_AutoClave  Model.Suffix_Fill1  Model.Suffix_Fill2
AJX75334501       AJX75334501             AJX75334501         AJX75334501           33820
AJX75334502       AJX75334502             AJX75334502         AJX75334502            3390
AJX75334505       AJX75334505             AJX75334505         AJX75334505            2635
AJX75334507       AJX75334507             AJX75334507         AJX75334507             310
AJX75334503       AJX75334503             AJX75334503         AJX75334503             162
AJX75334506       AJX75334506             AJX75334506         AJX75334506             129
AJX75334508       AJX75334508             AJX75334508         AJX75334508              60
Name: count, dtype: int64

In [18]:
# Model.Suffix_Dam의 이름을 Model.Suffix로 변경
train_data = train_data.rename(columns={'Model.Suffix_Dam': 'Model.Suffix'})
test_data = test_data.rename(columns={'Model.Suffix_Dam': 'Model.Suffix'})

# Model.Suffix_AutoClave, Model.Suffix_Fill1, Model.Suffix_Fill2 열 드롭
train_data = train_data.drop(columns=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2'])
test_data = test_data.drop(columns=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2'])

In [19]:
# Workorder로 시작하는 열 필터링
workorder_cols = ['Workorder_Dam', 'Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2']

# Workorder 열들로 이루어진 데이터프레임 생성
workorder_df = train_data[workorder_cols]

# 4개의 변수의 값이 모두 동일하지 않은 행 필터링
different_workorders = workorder_df[
    (workorder_df['Workorder_Dam'] != workorder_df['Workorder_AutoClave']) |
    (workorder_df['Workorder_Dam'] != workorder_df['Workorder_Fill1']) |
    (workorder_df['Workorder_Dam'] != workorder_df['Workorder_Fill2']) |
    (workorder_df['Workorder_AutoClave'] != workorder_df['Workorder_Fill1']) |
    (workorder_df['Workorder_AutoClave'] != workorder_df['Workorder_Fill2']) |
    (workorder_df['Workorder_Fill1'] != workorder_df['Workorder_Fill2'])
]

# 결과 출력
print("4개의 변수의 값이 모두 동일하지 않은 경우:")
print(different_workorders)

4개의 변수의 값이 모두 동일하지 않은 경우:
Empty DataFrame
Columns: [Workorder_Dam, Workorder_AutoClave, Workorder_Fill1, Workorder_Fill2]
Index: []


In [20]:
# Model.workorder_Dam의 이름을 Model.Suffix로 변경
train_data = train_data.rename(columns={'Workorder_Dam': 'Workorder'})
test_data = test_data.rename(columns={'Workorder_Dam': 'Workorder'})

# Model.Suffix_AutoClave, Model.Suffix_Fill1, Model.Suffix_Fill2 열 드롭
train_data = train_data.drop(columns=['Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'])
test_data = test_data.drop(columns=['Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'])

In [21]:
Process_Desc_col = train_data.filter(like='Insp. Seq No.').columns

new2_df = train_data.filter(items=Process_Desc_col)

new2_df.value_counts()

train_data = train_data.drop(columns=['Insp. Seq No._Dam', 'Insp. Seq No._AutoClave', 'Insp. Seq No._Fill1', 'Insp. Seq No._Fill2'])
test_data = test_data.drop(columns=['Insp. Seq No._Dam', 'Insp. Seq No._AutoClave', 'Insp. Seq No._Fill1', 'Insp. Seq No._Fill2'])

In [22]:
Process_Desc_col = train_data.filter(like='Insp Judge Code').columns

new2_df = train_data.filter(items=Process_Desc_col)

new2_df.value_counts()

train_data = train_data.drop(columns=['Insp Judge Code_Dam', 'Insp Judge Code_AutoClave', 'Insp Judge Code_Fill1', 'Insp Judge Code_Fill2'])
test_data = test_data.drop(columns=['Insp Judge Code_Dam', 'Insp Judge Code_AutoClave', 'Insp Judge Code_Fill1', 'Insp Judge Code_Fill2'])

In [24]:
train_data
test_data

,Set ID,Model.Suffix,Workorder,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,CURE STANDBY POSITION Z Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,...,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target,Equipment_same_num
0,0001be084fbc4aaa9d921f39e595961b,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,1150,33.5,0,...,270.0,50.0,85.0,19.8,13.0,195.0,1.0,0.0,NaN,1
1,0005bbd180064abd99e63f9ed3e1ac80,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,1150,33.5,0,...,270.0,50.0,85.0,19.8,14.0,256.0,1.0,NaN,NaN,1
2,000948934c4140d883d670adcb609584,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1150,33.5,0,...,270.0,50.0,85.0,19.7,1.0,98.0,1.0,0.0,NaN,1
3,000a6bfd02874c6296dc7b2e9c5678a7,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,1150,33.5,0,...,270.0,50.0,85.0,20.0,14.0,0.0,1.0,NaN,NaN,1
4,0018e78ce91343678716e2ea27a51c95,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1150,33.5,0,...,270.0,50.0,85.0,19.8,1.0,215.0,1.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,1150,33.5,0,...,270.0,50.0,85.0,19.5,14.0,131.0,1.0,NaN,NaN,1
17357,ffed8923c8a448a98afc641b770be153,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,1150,33.5,0,...,270.0,50.0,85.0,19.8,12.0,279.0,1.0,NaN,NaN,1
17358,fff1e73734da40adbe805359b3efb462,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1150,33.5,0,...,270.0,50.0,85.0,20.5,4.0,66.0,1.0,NaN,NaN,1
17359,fff8e38bdd09470baf95f71e92075dec,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1150,33.5,0,...,270.0,50.0,85.0,18.9,1.0,117.0,1.0,0.0,NaN,1


In [25]:
# 값의 종류가 1개이고 결측값이 없는 열을 제거하는 함수
def drop_single_value_columns(df):
    cols_to_drop = [col for col in df.columns if col != 'target' and df[col].nunique() == 1 and df[col].isnull().sum() == 0]
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped, cols_to_drop

# train_data와 test_data에서 해당 열 제거 및 삭제된 열 이름과 개수 출력
train_data, train_cols_dropped = drop_single_value_columns(train_data)
test_data, test_cols_dropped = drop_single_value_columns(test_data)

# print("삭제된 train_data 열 이름:", train_cols_dropped)
print("삭제된 train_data 열 개수:", len(train_cols_dropped))

# print("삭제된 test_data 열 이름:", test_cols_dropped)
print("삭제된 test_data 열 개수:", len(test_cols_dropped))

삭제된 train_data 열 개수: 42
삭제된 test_data 열 개수: 42


In [26]:
train_data.info()
print('---')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 119 entries, Model.Suffix to Equipment_same_num
dtypes: float64(58), int64(52), object(9)
memory usage: 36.8+ MB
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 120 entries, Set ID to Equipment_same_num
dtypes: float64(94), int64(17), object(9)
memory usage: 15.9+ MB


In [27]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 119 entries, Model.Suffix to Equipment_same_num
dtypes: float64(58), int64(52), object(9)
memory usage: 36.8+ MB


In [35]:
train_data.to_csv('train_data_fixed.csv', index=False)

In [29]:
# Step 1: Drop specified columns
columns_to_drop = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'WorkMode Collect Result_Dam',
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave',
    'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1',
    'WorkMode Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2',
    'WorkMode Collect Result_Fill2'
]

train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

In [30]:


# Step 2: Preprocess the Workorder variable (Keep only the first 5 characters)
train_data['Workorder'] = train_data['Workorder'].apply(lambda x: x[:5])
test_data['Workorder'] = test_data['Workorder'].apply(lambda x: x[:5])

# Step 3: Drop the Model.Suffix column
train_data = train_data.drop(columns=['Model.Suffix'])
test_data = test_data.drop(columns=['Model.Suffix'])

# Step 4: Drop identical columns from Dam process
columns_to_drop_dam = [
    'Stage1 Circle2 Distance Speed Collect Result_Dam',
    'Stage1 Circle3 Distance Speed Collect Result_Dam',
    'Stage1 Circle4 Distance Speed Collect Result_Dam',
    'Stage1 Line2 Distance Speed Collect Result_Dam',
    'Stage1 Line3 Distance Speed Collect Result_Dam',
    'Stage1 Line4 Distance Speed Collect Result_Dam',
    'Stage2 Circle2 Distance Speed Collect Result_Dam',
    'Stage2 Circle3 Distance Speed Collect Result_Dam',
    'Stage2 Circle4 Distance Speed Collect Result_Dam',
    'Stage2 Line2 Distance Speed Collect Result_Dam',
    'Stage2 Line3 Distance Speed Collect Result_Dam',
    'Stage2 Line4 Distance Speed Collect Result_Dam',
    'Stage3 Circle2 Distance Speed Collect Result_Dam',
    'Stage3 Circle3 Distance Speed Collect Result_Dam',
    'Stage3 Circle4 Distance Speed Collect Result_Dam',
    'Stage3 Line2 Distance Speed Collect Result_Dam',
    'Stage3 Line3 Distance Speed Collect Result_Dam',
    'Stage3 Line4 Distance Speed Collect Result_Dam'
]

train_data = train_data.drop(columns=columns_to_drop_dam)
test_data = test_data.drop(columns=columns_to_drop_dam)

# Step 5: Categorize the Chamber Temp. Collect Result_AutoClave variable in units of 5
train_data['Chamber Temp Category'] = pd.cut(train_data['Chamber Temp. Collect Result_AutoClave'], 
                                             bins=range(int(train_data['Chamber Temp. Collect Result_AutoClave'].min()), 
                                                        int(train_data['Chamber Temp. Collect Result_AutoClave'].max()) + 5, 5),
                                             right=False)

test_data['Chamber Temp Category'] = pd.cut(test_data['Chamber Temp. Collect Result_AutoClave'], 
                                            bins=range(int(test_data['Chamber Temp. Collect Result_AutoClave'].min()), 
                                                       int(test_data['Chamber Temp. Collect Result_AutoClave'].max()) + 5, 5),
                                            right=False)

# Step 6: Categorize the Chamber Temp. Unit Time_AutoClave variable in units of 50
train_data['Chamber Temp Unit Time Category'] = pd.cut(train_data['Chamber Temp. Unit Time_AutoClave'], 
                                                       bins=range(int(train_data['Chamber Temp. Unit Time_AutoClave'].min()), 
                                                                  int(train_data['Chamber Temp. Unit Time_AutoClave'].max()) + 50, 50),
                                                       right=False)

test_data['Chamber Temp Unit Time Category'] = pd.cut(test_data['Chamber Temp. Unit Time_AutoClave'], 
                                                      bins=range(int(test_data['Chamber Temp. Unit Time_AutoClave'].min()), 
                                                                 int(test_data['Chamber Temp. Unit Time_AutoClave'].max()) + 50, 50),
                                                      right=False)

In [32]:
# Dropping the specified columns from the train_data_final and test_data_final datasets
columns_to_drop_final = [
    'PalletID Collect Result_Dam',
    'Production Qty Collect Result_Dam',
    'Receip No Collect Result_Dam',
    'PalletID Collect Result_Fill2',
    'Production Qty Collect Result_Fill2'
]

train_data_final = train_data.drop(columns=columns_to_drop_final)
test_data_final = test_data.drop(columns=columns_to_drop_final)

# Displaying the new shapes of the datasets to confirm the columns were dropped
train_data_final.shape, test_data_final.shape

((40506, 89), (17361, 90))

In [33]:
# Listing the remaining columns in train_data_final to verify which ones are present
remaining_columns_train = train_data_final.columns.tolist()
remaining_columns_train

['Workorder',
 'CURE END POSITION X Collect Result_Dam',
 'CURE END POSITION Z Collect Result_Dam',
 'CURE END POSITION ? Collect Result_Dam',
 'CURE SPEED Collect Result_Dam',
 'CURE START POSITION X Collect Result_Dam',
 'CURE START POSITION ? Collect Result_Dam',
 'DISCHARGED SPEED OF RESIN Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
 'Dispense Volume(Stage1) Collect Result_Dam',
 'Dispense Volume(Stage2) Collect Result_Dam',
 'Dispense Volume(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
 'HEAD NO

In [36]:
# Grouping and categorizing the '1st Pressure 1st Pressure Unit Time_AutoClave' variable as specified
def categorize_pressure_unit_time(value):
    if value in [0, 1]:
        return 'Group 0-1'
    elif value in [60, 61]:
        return 'Group 60-61'
    elif value in [180, 181]:
        return 'Group 180-181'
    elif value in [240, 241]:
        return 'Group 240-241'
    else:
        return 'Other'

# Applying the function to create a new categorized variable
train_data_final['1st Pressure Unit Time Category'] = train_data_final['1st Pressure 1st Pressure Unit Time_AutoClave'].apply(categorize_pressure_unit_time)
test_data_final['1st Pressure Unit Time Category'] = test_data_final['1st Pressure 1st Pressure Unit Time_AutoClave'].apply(categorize_pressure_unit_time)

# Displaying the first few rows to confirm the new categorization
train_data_final[['1st Pressure 1st Pressure Unit Time_AutoClave', '1st Pressure Unit Time Category']].head()

# Grouping and categorizing the '2nd Pressure Unit Time_AutoClave' variable as specified
def categorize_2nd_pressure_unit_time(value):
    if value in [0, 1]:
        return 'Group 0-1'
    elif value in [60, 61]:
        return 'Group 60-61'
    elif value in [120, 121]:
        return 'Group 120-121'
    elif value in [180, 181]:
        return 'Group 180-181'
    else:
        return 'Other'

# Applying the function to create a new categorized variable
train_data_final['2nd Pressure Unit Time Category'] = train_data_final['2nd Pressure Unit Time_AutoClave'].apply(categorize_2nd_pressure_unit_time)
test_data_final['2nd Pressure Unit Time Category'] = test_data_final['2nd Pressure Unit Time_AutoClave'].apply(categorize_2nd_pressure_unit_time)

# Displaying the first few rows to confirm the new categorization
train_data_final[['2nd Pressure Unit Time_AutoClave', '2nd Pressure Unit Time Category']].head()

# Grouping and categorizing the '3rd Pressure Unit Time_AutoClave' variable as specified
def categorize_3rd_pressure_unit_time(value):
    if value in [60, 61]:
        return 'Group 60-61'
    elif value in [120, 121]:
        return 'Group 120-121'
    elif value in [180, 181]:
        return 'Group 180-181'
    else:
        return 'Other'

# Applying the function to create a new categorized variable
train_data_final['3rd Pressure Unit Time Category'] = train_data_final['3rd Pressure Unit Time_AutoClave'].apply(categorize_3rd_pressure_unit_time)
test_data_final['3rd Pressure Unit Time Category'] = test_data_final['3rd Pressure Unit Time_AutoClave'].apply(categorize_3rd_pressure_unit_time)

# Displaying the first few rows to confirm the new categorization
train_data_final[['3rd Pressure Unit Time_AutoClave', '3rd Pressure Unit Time Category']].head()

,3rd Pressure Unit Time_AutoClave,3rd Pressure Unit Time Category
0,120,Group 120-121
1,121,Group 120-121
2,121,Group 120-121
3,121,Group 120-121
4,120,Group 120-121


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [38]:
train_data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 92 columns):
 #   Column                                                      Non-Null Count  Dtype   
---  ------                                                      --------------  -----   
 0   Workorder                                                   40506 non-null  object  
 1   CURE END POSITION X Collect Result_Dam                      40506 non-null  int64   
 2   CURE END POSITION Z Collect Result_Dam                      40506 non-null  float64 
 3   CURE END POSITION ? Collect Result_Dam                      40506 non-null  int64   
 4   CURE SPEED Collect Result_Dam                               40506 non-null  int64   
 5   CURE START POSITION X Collect Result_Dam                    40506 non-null  int64   
 6   CURE START POSITION ? Collect Result_Dam                    40506 non-null  int64   
 7   DISCHARGED SPEED OF RESIN Collect Result_Dam                40506 non-null  

In [39]:
# Function to label encode all object-type columns in a DataFrame
def label_encode_object_columns(df):
    label_encoders = {}
    for column in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column].astype(str))
        label_encoders[column] = le
    return df, label_encoders

# Apply label encoding to train_data_final
train_data_final, train_label_encoders = label_encode_object_columns(train_data_final)

# Apply label encoding to test_data_final
test_data_final, test_label_encoders = label_encode_object_columns(test_data_final)


In [56]:
import xgboost as xgb
import re 
from sklearn.model_selection import cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split

In [57]:
# Step 1: Separate features (X) and target (y) in train_data_final
X = train_data_final.drop(columns=['target'])
y = train_data_final['target']

# Convert categorical columns to numeric using pd.get_dummies
X = pd.get_dummies(X)

# Clean the column names by replacing invalid characters
X.columns = [re.sub(r'[<>\[\]]', '_', col) for col in X.columns]

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Step 3: Define the parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
}

# Step 4: Set up Stratified K-Fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Step 5: Define the scoring metric
f1_scorer = make_scorer(f1_score)

# Step 6: Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    model, 
    param_distributions=param_grid, 
    n_iter=50,  # Number of parameter settings sampled
    scoring=f1_scorer, 
    cv=stratified_kfold, 
    verbose=1, 
    random_state=42,
    n_jobs=-1  # Use all available cores
)

In [58]:
X

,Workorder,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION ? Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION ? Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,...,"Chamber Temp Unit Time Category__53, 103)","Chamber Temp Unit Time Category__103, 153)","Chamber Temp Unit Time Category__153, 203)","Chamber Temp Unit Time Category__203, 253)","Chamber Temp Unit Time Category__253, 303)","Chamber Temp Unit Time Category__303, 353)","Chamber Temp Unit Time Category__353, 403)","Chamber Temp Unit Time Category__403, 453)","Chamber Temp Unit Time Category__453, 503)","Chamber Temp Unit Time Category__503, 553)"
0,103,240,2.5,-90,100,1030,-90,16,14.9,8.4,...,False,False,False,False,False,False,True,False,False,False
1,50,240,2.5,-90,70,1030,-90,10,21.3,4.9,...,False,False,False,False,False,False,False,False,True,False
2,95,1000,12.5,90,85,280,90,16,14.7,8.5,...,False,False,False,False,False,False,True,False,False,False
3,42,1000,12.5,90,70,280,90,10,21.3,8.4,...,False,False,False,False,False,False,False,False,True,False
4,26,240,2.5,-90,70,1030,-90,10,9.7,4.9,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,38,240,2.5,-90,70,1030,-90,10,17.0,5.0,...,False,False,False,False,False,False,False,False,True,False
40502,98,1000,12.5,90,100,280,90,16,14.9,8.5,...,False,False,False,False,False,False,True,False,False,False
40503,84,240,2.5,-90,100,1030,-90,16,14.2,8.2,...,False,False,False,False,False,False,True,False,False,False
40504,29,1000,12.5,90,70,280,90,10,9.7,4.9,...,False,False,False,False,False,False,False,False,True,False


In [59]:
# Step 7: Fit RandomizedSearchCV to the data
random_search.fit(X_train, y_train)

# Step 8: Print the best parameters and the corresponding F1 score
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best F1 Score: {random_search.best_score_}")

# The best model is now stored in random_search.best_estimator_
best_model = random_search.best_estimator_


Fitting 10 folds for each of 50 candidates, totalling 500 fits


c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:32:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'subsample': 0.6, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.2, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best F1 Score: 0.9713104778435152


In [60]:
# Step 9: Evaluate the model on the test set
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"F1 Score on the test set: {f1}")

F1 Score on the test set: 0.9718247153851046


In [61]:
# Step 1: Process test_data_final the same way as train_data_final
# Convert categorical columns to numeric using pd.get_dummies
X_test_final = pd.get_dummies(test_data_final)

# Ensure the columns in X_test_final match those in X_train
X_test_final = X_test_final.reindex(columns=X_train.columns, fill_value=0)

# Step 2: Use the best model to make predictions on the test_data_final
predictions = best_model.predict(X_test_final)


In [62]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [63]:
# Step 3: Save the predictions to a CSV file
submission = pd.DataFrame({
    'Set ID': test_data_final.index,  # Replace with the correct identifier if available
    'target': predictions
})

In [65]:
submission['target'].value_counts()

target
1    17309
0       52
Name: count, dtype: int64

In [66]:
# Mapping numeric predictions back to the original labels
target_mapping = {0: 'abnormal', 1: 'normal'}
submission['target'] = submission['target'].map(target_mapping)

In [67]:
submission

,Set ID,target
0,0,normal
1,1,normal
2,2,normal
3,3,normal
4,4,normal
...,...,...
17356,17356,normal
17357,17357,normal
17358,17358,normal
17359,17359,normal


In [68]:
# Ensure the Set ID is correctly added back to the submission file
submission['Set ID'] = test_data_final.index  # Replace with the correct identifier if available

# Reorder columns if necessary, typically 'Set ID' should be the first column
submission = submission[['Set ID', 'target']]

In [70]:
# Save the predictions to a CSV file
submission.to_csv('sub0810.csv', index=False)